# Análise de Sentimentos e Classificação de Textos em **PLN**

## Exemplos de Roteiro

### Exemplo 1 Análise de sentimentos com a biblioteca NLTK

In [1]:
# Importando as bibliotecas
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Certifique-se de baixar os recursos necessários do NLTK
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Inicializando o analisador de sentimentos
sid = SentimentIntensityAnalyzer()

# Definindo os textos de exemplo para treinar o modelo

textos = [
  "Este filme é incrível! Eu adorei cada minuto.",
  "Odiei este filme. Foi uma perda de tempo.",
  "O filme foi ok, nada de especial.",
  "Estou muito feliz com o produto, funciona perfeitamente!",
  "Que serviço horrível! Nunca mais compro nessa loja."
]

In [ ]:
# Analisando os sentimentos dos textos

for texto in textos:
  print(f"Texto: {texto}")
  ss = sid.polarity_scores(texto)
  print(f"Pontuaçao: {ss}")
  if ss['compound'] >= 0.05:
    print("Sentimento: Positivo")
  elif ss['compound' ] <= -0.05:
    print("Sentimento: Negativo")
  else:
    print("Sentimento: Neutro")
  print("\n")

Texto: Este filme é incrível! Eu adorei cada minuto.
Pontuaçao: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Odiei este filme. Foi uma perda de tempo.
Pontuaçao: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: O filme foi ok, nada de especial.
Pontuaçao: {'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'compound': 0.296}
Sentimento: Positivo


Texto: Estou muito feliz com o produto, funciona perfeitamente!
Pontuaçao: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Que serviço horrível! Nunca mais compro nessa loja.
Pontuaçao: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro




### Exemplo 2 Análise de sentimentos de Avaliações de Filmes com Naives Bayes

In [ ]:
# Primeira etapa: Criação de funções que calculam a quantidade de palavras, e as probabilidades

import numpy as np

def contar_palavras(avaliacoes, sentimento):
  """Conta a frequência de cada palavra em avaliações de um dado sentimento."""
  contagem = {}
  for avaliacao, sent in avaliacoes:
    if sent == sentimento:
      for palavra in avaliacao.lower().split():
        if palavra in contagem:
          contagem[palavra] += 1
        else:
          contagem[palavra] = 1

  return contagem

def calcular_probabilidade_sentimento(avaliacoes, sentimento):
  """Calcula a probabilidade de uma avaliação ser de um dado sentimento."""
  total_avaliacoes = len(avaliacoes)
  contagem_sentimento = sum(1 for _, sent in avaliacoes if sent == sentimento)
  return contagem_sentimento / total_avaliacoes

def calcular_probabilidade_palavra_dado_sentimento(palavra, contagem_palavras, total_palavras_sentimento, total_palavras_vocabulario, alpha=1):
  """Calcula a probabilidade de uma palavra aparecer dado um sentimento (com suavizacao Laplace)."""
  contagem_palavra = contagem_palavras.get(palavra, 0)
  return (contagem_palavra + alpha) / (total_palavras_sentimento + alpha * total_palavras_vocabulario)

In [ ]:
# Segunda etapa: Criação da função que calcula a classificação da palavra pertencer a um sentimento

def classificar_avaliacao(avaliacao, avaliacoes):
  """Classifica o sentimento de uma nova avaliaçao usando Naive Bayes."""

  sentimentos = ["Positivo", "Negativo"]
  probabilidades_sentimento = {}

  # Calcular probabilidades a priori dos sentimentos
  probabilidades_sentimento["Positivo"] = calcular_probabilidade_sentimento(avaliacoes, "Positivo")
  probabilidades_sentimento["Negativo"] = calcular_probabilidade_sentimento(avaliacoes, "Negativo")

  # Contar palavras em cada sentimento
  contagem_palavras_positivo = contar_palavras(avaliacoes, "Positivo")
  contagem_palavras_negativo = contar_palavras(avaliacoes, "Negativo")

  # Calcular total de palavras em cada sentimento
  total_palavras_positivo = sum(contagem_palavras_positivo.values())
  total_palavras_negativo = sum(contagem_palavras_negativo.values())

  # Calcular tamanho do vocabulário
  vocabulario = set()
  for avaliacao, _ in avaliacoes:
    for palavra in avaliacao.lower().split():
        vocabulario.add(palavra)
  total_palavras_vocabulario = len(vocabulario)

  # Calcular probabilidades para cada sentimento
  for sentimento in sentimentos:
      probabilidade_total = probabilidades_sentimento[sentimento]
      contagem_palavras_sentimento = contagem_palavras_positivo if sentimento == "Positivo" else contagem_palavras_negativo
      total_palavras_sentimento = total_palavras_positivo if sentimento == "Positivo" else total_palavras_negativo

      for palavra in avaliacao.lower().split():
          probabilidade_palavra = calcular_probabilidade_palavra_dado_sentimento(
            palavra, contagem_palavras_sentimento, total_palavras_sentimento, total_palavras_vocabulario
        )
          probabilidade_total *= probabilidade_palavra
      probabilidades_sentimento[sentimento] = probabilidade_total

  # Classificar com base na maior probabilidade
  melhor_sentimento = max(probabilidades_sentimento, key=probabilidades_sentimento.get)
  return melhor_sentimento

In [ ]:
# 	Terceira Etapa – Fornecimento de um modelo para treinamento

avaliacoes = [
    ("Filme incrível! Adorei a atuação.", "Positivo"),
    ("Roteiro fraco, não recomendo.", "Negativo"),
    ("Atuação boa, mas filme longo.", "Positivo"),
    ("Péssimo! Perdi meu tempo.", "Negativo"),
]


In [ ]:
# 	Quarta etapa  - Fornecendo um frase para ser avaliada

# Nova avaliação a ser classificada
nova_avaliacao = input("o que voce achou do filme 101 Dalmatas? ")

# Classificar a nova avaliação
sentimento = classificar_avaliacao(nova_avaliacao, avaliacoes)
print(f"A avaliação '{nova_avaliacao}' foi classificada como: {sentimento}")

o que voce achou do filme 101 Dalmatas? ótimo
A avaliação 'ótimo' foi classificada como: Positivo


In [ ]:
# Quinta etapa - Treine com um novo modelo de dados

avaliacoes = [
    ("Filme incrível! Adorei a atuação, roteiro e direção.", "Positivo"),
    ("Roteiro fraco, atuação ruim, não recomendo. Perda de tempo.", "Negativo"),
    ("Atuação boa, mas filme um pouco longo e arrastado.", "Positivo"),
    ("Péssimo! Perdi meu tempo, dinheiro e paciência. Horrível.", "Negativo"),
    ("Amei! Elenco maravilhoso, fotografia impecável. Recomendo muito!", "Positivo"),
    ("Chato, previsível e sem graça. Saí no meio do filme.", "Negativo"),
    ("Uma obra-prima! Roteiro inteligente, atuações memoráveis.", "Positivo"),
    ("Cansativo, atuações forçadas, final decepcionante.", "Negativo"),
    ("Divertido e emocionante! Ótimo para assistir com a família.", "Positivo"),
    ("Clichê, sem originalidade, diálogos fracos. Não vale a pena.", "Negativo"),
    ("Surpreendente e envolvente! Me prendeu do início ao fim.", "Positivo"),
    ("Confuso, mal dirigido, edição amadora. Um desastre.", "Negativo"),
    ("Atuação excelente, história cativante, trilha sonora perfeita.", "Positivo"),
    ("Atuações medianas, história arrastada, trilha sonora irritante.", "Negativo"),
    ("Roteiro bem construído, diálogos afiados, direção impecável.", "Positivo"),
    ("Roteiro fraco, diálogos bobos, direção perdida.", "Negativo"),
    ("Uma experiência cinematográfica única! Recomendo a todos os amantes do cinema.", "Positivo"),
    ("Um filme esquecível, sem emoção e sem impacto.", "Negativo"),
    ("Comovente e inspirador! Me fez refletir sobre a vida.", "Positivo"),
    ("Decepcionante e frustrante! Não recomendo nem para passar o tempo.", "Negativo"),
    ("Ótimo, gostei muito",'Positivo')
]

